## Importation des bibliothèques nécessaires

Dans cette cellule, nous importons toutes les bibliothèques indispensables au projet :

- `matplotlib` : pour l'affichage graphique des résultats.
- `dotenv` : pour charger des variables d'environnement si besoin.
- `warnings` : pour masquer les avertissements afin de garder le notebook lisible.
- `os` et `re` : pour manipuler les fichiers et gérer les noms.
- `requests` : utile en cas de téléchargement de données externes.
- `numpy`, `pandas` : pour le traitement des données.
- `PIL.Image` : pour manipuler les images.
- `torch` et `torch.nn.functional` : pour le deep learning avec PyTorch.
- `transformers` : pour charger le modèle SegFormer via HuggingFace.
- `tqdm` : pour afficher une barre de progression pendant les traitements longs.

In [3]:
%matplotlib inline
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')
import os
import requests
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from sklearn.metrics import confusion_matrix
from transformers import SegformerImageProcessor, AutoModelForSemanticSegmentation
import re
import time
from tqdm import tqdm

## Vérification de l'authentification Hugging Face

Dans cette cellule, nous :

1. Chargons les variables d'environnement grâce à `load_dotenv()` ;
2. Récupérons la clé API Hugging Face (`HF_API_KEY`) ;
3. Envoyons une requête à l'API Hugging Face (`/whoami-v2`) pour vérifier si le token est valide ;
4. Affichons un message de confirmation ou d'erreur en fonction de la réponse.

Cette étape est utile pour s'assurer que l'accès aux modèles hébergés sur Hugging Face est bien autorisé.


In [5]:
load_dotenv()

token = os.getenv("HF_API_KEY")

headers = {"Authorization": f"Bearer {token}"}
response = requests.get("https://huggingface.co/api/whoami-v2", headers=headers)

if response.status_code == 200:
    print("✅ Token valide !")
    print("Détails :", response.json())
else:
    print("❌ Token invalide ou expiré.")
    print("Code :", response.status_code)
    print("Message :", response.text)

✅ Token valide !
Détails : {'type': 'user', 'id': '67f75c19ee25083d34e674d2', 'name': 'JodyGs', 'fullname': 'Jody Gonzales', 'canPay': False, 'periodEnd': 1748735999, 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/no-auth/kPMbttmosNB83yyCC8RHE.png', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'p2_oc', 'role': 'fineGrained', 'createdAt': '2025-04-29T09:50:40.879Z', 'fineGrained': {'canReadGatedRepos': True, 'global': ['discussion.write', 'post.write'], 'scoped': [{'entity': {'_id': '67f75c19ee25083d34e674d2', 'type': 'user', 'name': 'JodyGs'}, 'permissions': ['repo.content.read', 'repo.write', 'inference.serverless.write', 'inference.endpoints.infer.write', 'inference.endpoints.write', 'user.webhooks.read', 'user.webhooks.write', 'collection.read', 'collection.write', 'discussion.write', 'user.billing.read']}]}}}}


## Chargement du modèle SegFormer préentraîné

Nous chargeons ici :

- **`SegformerImageProcessor`** : le préprocesseur associé au modèle, responsable du redimensionnement, de la normalisation et de la mise en forme des images d’entrée.
- **`AutoModelForSemanticSegmentation`** : le modèle de segmentation sémantique SegFormer, ici spécialisé dans la détection de vêtements (`segformer_b2_clothes`).

Ces composants sont fournis via l’API de Hugging Face, ce qui permet de bénéficier de modèles d’État de l’art déjà entraînés sur des datasets pertinents.


In [12]:
processor = SegformerImageProcessor.from_pretrained("mattmdjaga/segformer_b2_clothes")
model = AutoModelForSemanticSegmentation.from_pretrained("mattmdjaga/segformer_b2_clothes")

## Définition des dossiers et des classes de segmentation

### Dossiers utilisés :
- `assets/IMG` : dossier contenant les images à segmenter.
- `assets/pred_masks` : dossier de sauvegarde des masques prédits (format brut et coloré).
- `assets/true_masks` : dossier contenant les masques de vérité terrain (ground truth).
- `assets/visu_masks` : dossier de sauvegarde des visualisations côte à côte (image originale + image segmentée).

Les dossiers de sortie sont automatiquement créés s’ils n’existent pas.

### Classes de segmentation :
Le dictionnaire `id2label` définit les 18 classes possibles dans la segmentation (vêtements, accessoires, parties du corps, etc.).

### Palette de couleurs :
Une couleur RGB unique est associée à chaque classe, afin de produire des masques colorés lisibles visuellement.


In [13]:
image_folder = "assets/IMG"
pred_mask_folder = "assets/pred_masks"
true_mask_folder = "assets/true_masks"
os.makedirs(pred_mask_folder, exist_ok=True)
visu_mask_folder = "assets/visu_masks"
os.makedirs(visu_mask_folder, exist_ok=True)

# Labels
id2label = {
    0: "Background", 1: "Hat", 2: "Hair", 3: "Sunglasses", 4: "Upper-clothes",
    5: "Skirt", 6: "Pants", 7: "Dress", 8: "Belt", 9: "Left-shoe", 10: "Right-shoe",
    11: "Face", 12: "Left-leg", 13: "Right-leg", 14: "Left-arm", 15: "Right-arm",
    16: "Bag", 17: "Scarf"
}
num_classes = len(id2label)

# Couleurs associées (palette RGB)
color_palette = [
    (0, 0, 0), (255, 0, 0), (255, 165, 0), (255, 255, 0), (0, 128, 0),
    (0, 255, 255), (0, 0, 255), (138, 43, 226), (128, 0, 0), (255, 192, 203),
    (199, 21, 133), (160, 82, 45), (70, 130, 180), (100, 149, 237),
    (127, 255, 212), (0, 100, 0), (169, 169, 169), (75, 0, 130)
]

## 🧠 Boucle de traitement des images et génération des masques

Cette boucle parcourt toutes les images du dossier `assets/IMG` pour effectuer les étapes suivantes :

1. **Chargement et préparation** :
   - Chaque image est redimensionnée à `512x512` pixels et convertie en RGB.
   - Elle est ensuite prétraitée pour être compatible avec le modèle SegFormer.

2. **Prédiction du masque** :
   - Le modèle prédit une carte de segmentation (`logits`), qui est interpolée à la bonne taille.
   - Le masque final est obtenu par un `argmax` sur les logits pour chaque pixel.

3. **Sauvegarde des résultats** :
   - Le masque brut (grayscale, une valeur par pixel correspondant à une classe) est sauvegardé.
   - Un masque coloré (RGB) est généré à partir de la palette définie.
   - Un masque transparent (RGBA) est superposé sur l’image originale.

4. **Visualisation** :
   - Une figure contenant l’image originale et l’image segmentée côte à côte est affichée avec une légende dynamique.
   - Cette figure est également sauvegardée dans `assets/visu_masks`.

5. **Pause toutes les 30 images** :
   - Pour éviter la surchauffe ou les erreurs système, une pause de 5 secondes est effectuée toutes les 30 images traitées.


In [14]:
import re  # pour extraire les indices numériques

# Traitement de chaque image du dossier
for i, filename in enumerate(sorted(os.listdir(image_folder))):
    if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    print(f"Traitement de : {filename}")
    image_path = os.path.join(image_folder, filename)
    image = Image.open(image_path).convert("RGB").resize((512, 512))

    # Prétraitement
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits.cpu()

    upsampled_logits = F.interpolate(
        logits, size=(512, 512), mode="bilinear", align_corners=False
    )
    pred_seg = upsampled_logits.argmax(dim=1)[0].numpy()

    print("Valeurs uniques prédites :", np.unique(pred_seg))

    match = re.search(r'\d+', filename)
    index = match.group() if match else os.path.splitext(filename)[0]
    base_name = f"mask_{index}"

    # Sauvegarde des masques
    raw_path = os.path.join(pred_mask_folder, base_name + ".png")
    Image.fromarray(pred_seg.astype(np.uint8), mode="L").save(raw_path)

    rgb_mask = np.zeros((*pred_seg.shape, 3), dtype=np.uint8)
    for class_id in np.unique(pred_seg):
        if class_id < len(color_palette):
            rgb_mask[pred_seg == class_id] = color_palette[class_id]
    Image.fromarray(rgb_mask).save(os.path.join(pred_mask_folder, base_name + "_color.png"))

    # Création d’un masque transparent superposable
    color_mask = np.zeros((*pred_seg.shape, 4), dtype=np.uint8)
    for class_id in np.unique(pred_seg):
        if class_id < len(color_palette):
            color = color_palette[class_id]
            mask = pred_seg == class_id
            color_mask[mask] = (*color, 120)

    blended = Image.alpha_composite(image.convert("RGBA"), Image.fromarray(color_mask))

    # Affichage et sauvegarde côte à côte
    fig, axs = plt.subplots(1, 2, figsize=(14, 6))
    axs[0].imshow(image)
    axs[0].set_title("Image originale")
    axs[0].axis("off")

    axs[1].imshow(blended)
    axs[1].set_title("Image segmentée")
    axs[1].axis("off")

    handles = [
        plt.Line2D([0], [0], marker='o', color='w',
                   markerfacecolor=np.array(color_palette[c])/255,
                   label=id2label[c], markersize=10)
        for c in np.unique(pred_seg) if c in id2label
    ]
    fig.legend(handles=handles, bbox_to_anchor=(1.05, 0.5), loc="center left")
    plt.tight_layout()

    # Enregistrement dans le dossier visu_masks
    visupath = os.path.join(visu_mask_folder, base_name + "_visu.png")
    plt.savefig(visupath)
    plt.close(fig)

    # Pause toutes les 30 images
    if (i + 1) % 30 == 0:
        print("Pause de 5 secondes après 30 images...")
        time.sleep(5)


📷 Traitement de : image_0.png
Valeurs uniques prédites : [ 0  2  4  6  9 10 11 12 13 15 16]
📷 Traitement de : image_1.png
Valeurs uniques prédites : [ 0  1  2  3  4  5  6  9 10 11 16]
📷 Traitement de : image_10.png
Valeurs uniques prédites : [ 0  2  4  5  8  9 10 11 12 13 14 15 16]
📷 Traitement de : image_11.png
Valeurs uniques prédites : [ 0  1  2  4  5  9 10 11 12 13 14 15]
📷 Traitement de : image_12.png
Valeurs uniques prédites : [ 0  2  4  6  9 10 11 12 13 14 15]
📷 Traitement de : image_13.png
Valeurs uniques prédites : [ 0  2  4  5  6  8  9 10 11 15 16]
📷 Traitement de : image_14.png
Valeurs uniques prédites : [ 0  2  4  6  8  9 10 11 12 13 14 15 16]
📷 Traitement de : image_15.png
Valeurs uniques prédites : [ 0  2  4  6  9 10 11 12 13 14 15]
📷 Traitement de : image_16.png
Valeurs uniques prédites : [ 0  2  7  9 10 11 12 13 14 15 16]
📷 Traitement de : image_17.png
Valeurs uniques prédites : [ 0  2  3  4  6  9 10 11 12 13 14 15 16]
📷 Traitement de : image_18.png
Valeurs uniques préd

## Évaluation des performances du modèle

Dans cette cellule, nous évaluons la qualité des segmentations produites par le modèle en comparant les masques prédits aux masques de vérité terrain (`true_masks`).

### Étapes détaillées :

1. **Construction de la matrice de confusion** :
   - Pour chaque paire masque réel / masque prédit, on calcule les correspondances de classes au pixel près.
   - La matrice de confusion permet de savoir combien de fois chaque classe a été correctement ou incorrectement prédite.

2. **Calcul des métriques par classe** :
   - **IoU (Intersection over Union)** : mesure le chevauchement entre le masque prédit et le masque réel.
   - **Accuracy** : proportion de pixels bien classés pour chaque classe.
   - **Dice Score** : alternative à l’IoU, souvent utilisée en segmentation médicale, qui pondère différemment les faux positifs et les faux négatifs.

3. **Tableau récapitulatif** :
   - Chaque classe est évaluée individuellement.
   - Une ligne "MOYENNE" synthétise les performances globales du modèle.

Ces indicateurs sont essentiels pour juger de la robustesse du modèle sur l’ensemble des catégories segmentées.


In [15]:
# Matrice de confusion
conf_matrix = np.zeros((num_classes, num_classes), dtype=np.uint64)

for filename in os.listdir(true_mask_folder):
    if not filename.endswith(".png"):
        continue

    true_path = os.path.join(true_mask_folder, filename)
    pred_path = os.path.join(pred_mask_folder, filename)

    if not os.path.exists(pred_path):
        continue

    true_mask = np.array(Image.open(true_path).resize((512, 512)))
    pred_mask = np.array(Image.open(pred_path).resize((512, 512)))

    if true_mask.shape != pred_mask.shape:
        continue

    conf_matrix += confusion_matrix(
        true_mask.flatten(), pred_mask.flatten(), labels=list(range(num_classes))
    ).astype(np.uint64)

# Calcul IoU, Accuracy, Dice
ious, accuracies, dice_scores = [], [], []
for i in range(num_classes):
    TP = conf_matrix[i, i]
    FP = conf_matrix[:, i].sum() - TP
    FN = conf_matrix[i, :].sum() - TP

    denom_iou = TP + FP + FN
    iou = TP / denom_iou if denom_iou > 0 else 0.0
    ious.append(round(iou, 2))

    denom_acc = conf_matrix[i, :].sum()
    acc = TP / denom_acc if denom_acc > 0 else 0.0
    accuracies.append(round(acc, 2))

    denom_dice = 2 * TP + FP + FN
    dice = 2 * TP / denom_dice if denom_dice > 0 else 0.0
    dice_scores.append(round(dice, 2))

# Tableau final avec moyenne
mean_dice = round(np.mean(dice_scores), 3)
mean_iou = round(np.mean(ious), 3)
mean_acc = round(np.mean(accuracies), 3)

df_iou = pd.DataFrame({
    "Label Index": list(id2label.keys()),
    "Label Name": list(id2label.values()),
    "Category Accuracy": accuracies,
    "Category IoU": ious,
    "Dice Score": dice_scores
}).sort_values("Category IoU", ascending=False).reset_index(drop=True)

df_iou.loc[len(df_iou.index)] = [
    "—", "MOYENNE", mean_acc, mean_iou, mean_dice
]

# Affichage final
print(f"✅ Dice Score moyen : {mean_dice}")
print(f"✅ mIoU globale : {mean_iou}")
display(df_iou)

✅ Dice Score moyen : 0.796
✅ mIoU globale : 0.688


Label Index     Label Name  Category Accuracy  Category IoU  Dice Score
0            0     Background              0.990         0.990       0.990
1            4  Upper-clothes              0.960         0.890       0.940
2            6          Pants              0.960         0.890       0.940
3            7          Dress              0.900         0.860       0.930
4            5          Skirt              0.880         0.830       0.910
5           12       Left-leg              0.870         0.780       0.870
6           11           Face              0.880         0.770       0.870
7            2           Hair              0.850         0.750       0.860
8           13      Right-leg              0.860         0.750       0.860
9           15      Right-arm              0.890         0.740       0.850
10          16            Bag              0.860         0.720       0.840
11          14       Left-arm              0.860         0.680       0.810
12          17          Scarf              0.780         0.660       0.800
13          10     Right-shoe              0.680         0.540       0.700
14           1            Hat              0.540         0.520       0.680
15           9      Left-shoe              0.640         0.510       0.670
16           8           Belt              0.340         0.310       0.480
17           3     Sunglasses              0.200         0.190       0.320
18           —        MOYENNE              0.774         0.688       0.796